In [ ]:
# !pip install transformers
# !pip install torch
# !pip install tensorflow

In [1]:
from huggingface_hub import login, logout
login()
# Should see this:
# Token is valid (permission: write).
# Your token has been saved in your configured git credential helpers (manager).
# Your token has been saved to C:\Users\Owner\.cache\huggingface\token
# Login successful

In [4]:
# following along with the Datacamp tutorial here:
# https://www.datacamp.com/datalab/w/be6d48e3-3eb3-417c-8787-595d0636535e/edit
import os
import torch
import huggingface_hub as hf_hub
import datasets
import transformers

In [5]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")
# Print the tokenizer to see the data preprocessing configuration for this model 
print(tokenizer)

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

C:\VirtualEnvironments\datacamp\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Owner\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-emoji. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

RobertaTokenizerFast(name_or_path='cardiffnlp/twitter-roberta-base-emoji', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}


## What from_pretrained() does

The from_pretrained() method (see [docs](https//huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.from_pretrained)) first searches for a model repository with the same name on the Hugging Face Hub but it also accepts a local path or a URL with the expected folder structure. You can simply git clone the repository and load it from your local path.

## Tokenizers

NLP models can't process text inputs as it is and need to be converted to a fixed length mathematical format. The Tokenizer classes preprocess the text such that each word and punctuation is given a unique ID or a token, short sentences are padded and long sentences are truncated to create fixed-size input vectors. They also allow using additional tokens such as bos, eos, unk, sep, and more to specify start and end of sentences, and to assign token IDS to unknown words that are not in the tokenizer vocabulary.

The output of the tokenizer tells us this pretrained model uses a tokenizer with a 50265 unique token IDs that applies padding or truncation to the end of each input text, and removes leading (leftside) extra whitespaces. You can always refer to the corresponding Tokenizer docs
to learn more about each preprocessing step.

In [6]:
# Load the pre-trained "cardiffnlp/twitter-roberta-base-emoji" model
model = AutoModel.from_pretrained("cardiffnlp/twitter-roberta-base-emoji")

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emoji and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Notes on Model Loading

Take a look at the warning above. We were only able to load a chunk of the model parameters included in the checkpoint with the AutoModel class.

This is because all transformers models are designed to have a single base class and multiple task-specific prediction classes built on top of it. The AutoModel class is designed to only load the base model parameters such as `RobertaModel` but not task-specific models such as `RobertaForSequenceClassification`.

In order to identify the exact class name and task of your target checkpoint, you can simply refer to the model configuration.